In [1]:
import sqlite3
import math
from typing import Dict, List, Union, Any, Optional, Set, Tuple
import pandas as pd
import numpy as np
import os, time, re
from collections import defaultdict


In [2]:
# ===== CONFIG =====
MAGI_DB_PATH = os.getenv("MAGI_DB_PATH", "/projects/klybarge/pcori_ad/magi/magi_db/magi.db")  # read-only; override via env
OUT_DIR = "./MAGI_LASSO"
os.makedirs(OUT_DIR, exist_ok=True)
EDGE_TABLE = "magi_counts_top500"   
uri = f"file:{MAGI_DB_PATH}?mode=ro"

TOP_K = 1900  # 500 for others


In [3]:
TARGETS = [
    "aa_meas_amitriptyline_rem",
    "aa_meas_fluoxetine_rem", 
    "aa_meas_citalopram_rem",
    "aa_meas_venlafaxine_rem",
    "aa_meas_mirtazapine_rem",
    "aa_meas_sertraline_rem",
    "aa_meas_bupropion_rem",
    "aa_meas_trazodone_rem",
    "aa_meas_duloxetine_rem",
    "aa_meas_escitalopram_rem",
    "aa_meas_paroxetine_rem",
    "aa_meas_nortriptyline_rem",
    "aa_meas_other_rem",
    "aa_meas_doxepin_rem",
    "aa_meas_desvenlafaxine_rem",
]



In [4]:


TARGETS = [
    "aa_meas_citalopram_rem",
]

In [5]:


## Update 10 15 25
def analyze_causal_sequence_py(
    df_in: Union[str, pd.DataFrame],
    *,
    name_map: Dict[str, str] = None,     # raw_code -> friendly label (applied to BOTH columns)
    events: List[str] = None,            # event names to KEEP (AFTER recoding). If None: auto-detect
    force_outcome: str = None,           # if provided and present, force this to be the FINAL node (Y)
    lambda_min_count: int = 15           # L-threshold for λ: if n_code < L ⇒ λ_{k,j}=0
) -> Dict[str, Any]:
    """
    MAGI (Python): Reference routine with explicit comments.

    ─────────────────────────────────────────────────────────────────────────────
    COLUMN CONVENTION PER ROW:
      Left  column: target_concept_code  (call this X for this row)
      Right column: concept_code         (call this k for this row)

      n_code_target        ≡  k ∧ X
      n_code_no_target     ≡  k ∧ ¬X
      n_target_no_code     ≡  ¬k ∧ X
      n_no_target          ≡  total(¬X)
      n_target             ≡  total(X)
      n_code               ≡  total(k)
      n_code_before_target ≡  count(k before X)
      n_target_before_code ≡  count(X before k)

    ORIENTATION (locked to your spec):
      • Total effect T_{kY}:   read row (target = Y, code = k).
      • Lambda     λ_{k,j}:    read row (target = j, code = k), and compute
                               λ_{k,j} = n_code_target / n_code  with L-threshold on n_code.

    TEMPORAL SCORE for each node Zi:
      Score(Z_i) = Σ_{j≠i} [ C(Z_i≺Z_j) - C(Z_j≺Z_i) + C(Z_i∧¬Z_j) - C(Z_j∧¬Z_i) ]
      Read from row (target=Z_j, code=Z_i):
        - n_code_before_target      → C(Z_i≺Z_j)
        - n_target_before_code      → C(Z_j≺Z_i)
        - n_code_no_target          → C(Z_i∧¬Z_j)
        - n_target_no_code          → C(Z_j∧¬Z_i)

    T_{kY} from row (Y, k):
      a = n_code_target        (k ∧ Y)
      b = n_code_no_target     (k ∧ ¬Y)
      c = n_target_no_code     (¬k ∧ Y)
      d = n_no_target - b      (¬k ∧ ¬Y)   ← computed on the fly (no extra column needed)
      With sample-size–anchored odds:
         odds_k1 = a/b with guards; odds_k0 = c/d with guards; T = odds_k1 / odds_k0

    DIRECT EFFECTS via backward recursion:
      D_{k,Y} = ( T_{k,Y} - Σ_j λ_{k,j} D_{j,Y} ) / ( 1 - Σ_j λ_{k,j} ),
      where j are downstream nodes between k and Y in the temporal order.

    LOGISTIC LINK:
      logit P(Y=1 | Z) = β0 + Σ_k β_k Z_k  with β_k = log D_{k,Y};
      invalid/nonpositive D map to β_k=0.

    RETURNS a dict with:
      sorted_scores, temporal_order, order_used,
      T_val, D_val, lambda_l, coef_df, beta_0, beta, logit_predictors, predict_proba
    """

    # ── 0) Ingest & validate ───────────────────────────────────────────────────
    df = pd.read_csv(df_in) if isinstance(df_in, str) else df_in.copy()

    need_cols = [
        "target_concept_code", "concept_code",
        "n_code_target", "n_code_no_target",
        "n_target", "n_no_target",
        "n_target_no_code",
        "n_code",                                 # for λ denominator
        "n_code_before_target", "n_target_before_code",
    ]
    missing = [c for c in need_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {', '.join(missing)}")

    # Optional recoding to friendly labels (applied to BOTH endpoints)
    if name_map:
        df["target_concept_code"] = df["target_concept_code"].replace(name_map)
        df["concept_code"]        = df["concept_code"].replace(name_map)

    # Limit to selected events (union if auto)
    if events is None:
        ev_t = df["target_concept_code"].astype(str).unique().tolist()
        ev_c = df["concept_code"].astype(str).unique().tolist()
        events = sorted(set(ev_t) | set(ev_c))
    if len(events) < 2:
        raise ValueError("Need at least two events.")

    df = df[df["target_concept_code"].isin(events) & df["concept_code"].isin(events)].copy()

    # Coerce numerics; NA→0 to allow safe sums/max
    num_cols = [
        "n_code_target", "n_code_no_target",
        "n_target", "n_no_target", "n_target_no_code",
        "n_code", "n_code_before_target", "n_target_before_code",
    ]
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)

    # ── 1) Temporal score (read rows: target=Z_j, code=Z_i) ────────────────────
    scores: Dict[str, float] = {}
    for zi in events:
        s = 0.0
        for zj in (x for x in events if x != zi):
            # Row oriented as (Zj, Zi)
            pair = df[(df["target_concept_code"] == zj) & (df["concept_code"] == zi)]
            if pair.empty:
                continue
            c_i_before_j = float(pair["n_code_before_target"].sum(skipna=True))   # Zi before Zj
            c_j_before_i = float(pair["n_target_before_code"].sum(skipna=True))   # Zj before Zi
            c_i_and_not_j = float(pair["n_code_no_target"].sum(skipna=True))      # Zi ∧ ¬Zj
            c_j_and_not_i = float(pair["n_target_no_code"].sum(skipna=True))      # Zj ∧ ¬Zi
            s += (c_i_before_j - c_j_before_i + c_i_and_not_j - c_j_and_not_i)
        scores[zi] = s

    sorted_scores = pd.Series(scores).sort_values(ascending=False)

    # Choose outcome Y: either forced or top-scoring node
    if force_outcome and (force_outcome in sorted_scores.index):
        outcome = force_outcome
        temporal_order = [ev for ev in sorted_scores.index if ev != outcome] + [outcome]
    else:
        outcome = sorted_scores.index[0]
        temporal_order = [ev for ev in sorted_scores.index if ev != outcome] + [outcome]

    events_order = temporal_order              # earliest … → Y
    antecedents = events_order[:-1]            # everything before Y

    # ── 2) T and λ (row orientations locked) ───────────────────────────────────
    T_val = pd.Series(0.0, index=antecedents, dtype=float)
    D_val = pd.Series(np.nan, index=antecedents, dtype=float)
    lambda_l: Dict[str, pd.Series] = {}

    for k in antecedents:
        # ---- T_{kY} from the single row (target=Y, code=k) ----
        row_Yk = df[(df["target_concept_code"] == outcome) & (df["concept_code"] == k)]

        # 2×2 cells (a,b,c,d) at (Y,k):
        a = float(row_Yk["n_code_target"].sum(skipna=True))        # k ∧ Y
        b = float(row_Yk["n_code_no_target"].sum(skipna=True))     # k ∧ ¬Y
        c = float(row_Yk["n_target_no_code"].sum(skipna=True))     # ¬k ∧ Y
        # d is not in data; compute from the same row: d = total(¬Y) − (k ∧ ¬Y)
        n_noY = float(row_Yk["n_no_target"].max(skipna=True)) if not row_Yk.empty else 0.0
        d = max(n_noY - b, 0.0)                                    # ¬k ∧ ¬Y

        # Stratum sizes
        N1 = a + b                   # k = 1
        N0 = c + d                   # k = 0

        # Sample-size–anchored odds for k=1 and k=0
        if N1 == 0:
            odds_k1 = 1.0
        else:
            if a == 0:
                odds_k1 = 1.0 / (N1 + 1.0)
            elif b == 0:
                odds_k1 = (N1 + 1.0)
            else:
                odds_k1 = a / b

        if N0 == 0:
            odds_k0 = 1.0
        else:
            if c == 0:
                odds_k0 = 1.0 / (N0 + 1.0)
            elif d == 0:
                odds_k0 = (N0 + 1.0)
            else:
                odds_k0 = c / d

        T_val.loc[k] = float(odds_k1 / odds_k0) if odds_k0 > 0 else (N1 + 1.0)

        # ---- λ_{k,j} from rows (target=j, code=k): λ = n_code_target / n_code ----
        pos_k = events_order.index(k)
        js = events_order[pos_k + 1 : -1] if pos_k < len(events_order) - 1 else []

        lam_pairs = []
        for j in js:
            row_jk = df[(df["target_concept_code"] == j) & (df["concept_code"] == k)]
            if row_jk.empty:
                lam_pairs.append((j, 0.0))
                continue

            num = float(pd.to_numeric(row_jk["n_code_target"], errors="coerce").fillna(0.0).sum())
            den = float(pd.to_numeric(row_jk["n_code"],        errors="coerce").fillna(0.0).sum())

            # L-threshold on the conditioning size n_code (count of k)
            if (den <= 0) or (den < lambda_min_count):
                lam_pairs.append((j, 0.0))
                continue

            lam = num / den
            lam = 0.0 if not np.isfinite(lam) else float(min(max(lam, 0.0), 1.0))
            lam_pairs.append((j, lam))

        lambda_l[k] = pd.Series({j: v for j, v in lam_pairs}, dtype=float)

    # ── 3) Backward recursion for direct effects D ─────────────────────────────
    # Last antecedent (just before Y): no downstream → D = T
    if len(antecedents) >= 1:
        last_anc = antecedents[-1]
        D_val.loc[last_anc] = T_val.loc[last_anc]

    # Walk backward for the rest
    if len(antecedents) > 1:
        for k in list(reversed(antecedents[:-1])):
            lam_vec = lambda_l.get(k, pd.Series(dtype=float))
            downstream = list(lam_vec.index)  # j nodes after k (already resolved)
            lam_vals = lam_vec.reindex(downstream).fillna(0.0).to_numpy()
            D_down  = pd.to_numeric(D_val.reindex(downstream), errors="coerce").fillna(0.0).to_numpy()

            num = T_val.loc[k] - float(np.nansum(lam_vals * D_down))
            den = 1.0 - float(np.nansum(lam_vals))

            if (not np.isfinite(den)) or den == 0.0:
                D_val.loc[k] = T_val.loc[k]            # neutralize if pathological
            else:
                tmp = num / den
                D_val.loc[k] = tmp if np.isfinite(tmp) else T_val.loc[k]

    # ── 4) Logistic link (β) and predict_proba ─────────────────────────────────
    # Intercept β0 from marginal prevalence of Y (rows with target == Y)
    resp_rows = df[df["target_concept_code"] == outcome]
    n_t = float(resp_rows["n_target"].max(skipna=True)) if not resp_rows.empty else np.nan
    n_n = float(resp_rows["n_no_target"].max(skipna=True)) if not resp_rows.empty else np.nan
    denom = n_t + n_n
    p_y = 0.5 if (not np.isfinite(denom) or denom <= 0) else (n_t / denom)
    beta_0 = float(np.log(p_y / (1 - p_y)))

    # β_k = log D_{k,Y}; invalid/nonpositive → 0
    D_clean = pd.to_numeric(D_val, errors="coerce").astype(float)
    beta_vals = np.log(D_clean.where(D_clean > 0.0)).replace([np.inf, -np.inf], np.nan).fillna(0.0)

    coef_df = pd.DataFrame({
        "predictor": list(beta_vals.index) + ["(intercept)"],
        "beta":      list(beta_vals.values) + [beta_0],
    })

    # Vectorized predict_proba
    predictors = list(beta_vals.index)
    beta_vec = beta_vals.values

    def predict_proba(Z: Union[Dict[str, Any], pd.Series, np.ndarray, List[float], pd.DataFrame]):
        """
        Compute P(Y=1|Z) using: logit P = β0 + Σ_k β_k Z_k.
        Z can be:
          - dict/Series mapping predictor name -> 0/1
          - 1D/2D numpy/list with columns ordered as `predictors`
          - DataFrame containing any/all of `predictors` (others ignored)
        """
        def sigmoid(x):
            x = np.clip(x, -700, 700)  # numerical stability for large |η|
            return 1.0 / (1.0 + np.exp(-x))

        if isinstance(Z, pd.DataFrame):
            M = Z.reindex(columns=predictors, fill_value=0.0).astype(float).to_numpy()
            return sigmoid(beta_0 + M @ beta_vec)

        if isinstance(Z, (dict, pd.Series)):
            v = np.array([float(Z.get(p, 0.0)) for p in predictors], dtype=float)
            return float(sigmoid(beta_0 + float(v @ beta_vec)))

        arr = np.asarray(Z, dtype=float)
        if arr.ndim == 1:
            if arr.size != len(predictors):
                raise ValueError(f"Expected {len(predictors)} features in order: {predictors}")
            return float(sigmoid(beta_0 + float(arr @ beta_vec)))
        if arr.ndim == 2:
            if arr.shape[1] != len(predictors):
                raise ValueError(f"Expected shape (*,{len(predictors)}), got {arr.shape}")
            return sigmoid(beta_0 + arr @ beta_vec)

        raise ValueError("Unsupported input for predict_proba")

    # ── 5) Package results ─────────────────────────────────────────────────────
    return {
        "sorted_scores": sorted_scores,
        "temporal_order": temporal_order,
        "order_used": events_order,
        "T_val": T_val,
        "D_val": D_val,
        "lambda_l": lambda_l,
        "coef_df": coef_df,
        "beta_0": beta_0,
        "beta": pd.Series(beta_vec, index=predictors, dtype=float),
        "logit_predictors": predictors,
        "predict_proba": predict_proba,
    }

def _ensure_derived_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    Only coerce numerics and compute total_effect if absent, using provided cells and
    computing d = n_no_target - n_code_no_target on the fly (since d isn't in data).
    """
    required = [
        "n_code_target", "n_code_no_target",
        "n_target_no_code", "n_target", "n_no_target"
    ]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns for TE: {', '.join(missing)}")

    out = df.copy()
    for c in required:
        out[c] = pd.to_numeric(out[c], errors="coerce").fillna(0.0)

    if "total_effect" not in out.columns:
        a = out["n_code_target"].astype(float)
        b = out["n_code_no_target"].astype(float)
        c = out["n_target_no_code"].astype(float)
        d = (out["n_no_target"] - out["n_code_no_target"]).astype(float)  # compute d locally

        N1 = a + b
        N0 = c + d

        with np.errstate(divide="ignore", invalid="ignore"):
            odds_k1 = np.where(
                N1 == 0, 1.0,
                np.where((a > 0) & (b > 0), a / b,
                         np.where(b == 0, N1 + 1.0, 1.0 / (N1 + 1.0)))
            )
            odds_k0 = np.where(
                N0 == 0, 1.0,
                np.where((c > 0) & (d > 0), c / d,
                         np.where(d == 0, N0 + 1.0, 1.0 / (N0 + 1.0)))
            )
            te = odds_k1 / odds_k0

        out["total_effect"] = np.where(np.isfinite(te), te, 1.0).astype(float)

    return out


# TOPK_HI/LO
def _select_top_by_te_unique_k(k_to_T: pd.DataFrame, top_k: int = TOP_K,
                               hi: float = 1.5) -> pd.DataFrame:
    """
    Balanced selection without fallback:
      - Take up to top_k//2 strongest risk (TE >= hi)
      - Take up to top_k - top_k//2 strongest protective (TE <= 1/hi)
      - If either side is short, do NOT fill from elsewhere; total may be < top_k
      - Return one row per k (**concept_code**), ranked by extremeness
    """
    df = k_to_T.copy()

    # Ensure total_effect numeric
    df["total_effect"] = pd.to_numeric(df["total_effect"], errors="coerce")
    df = df.dropna(subset=["total_effect"]).copy()
    if df.empty:
        print("[SELECT] no rows with total_effect; returning empty selection.")
        return df

    # Extremeness symmetrical around 1
    df["effect_strength"] = np.where(df["total_effect"] >= 1.0,
                                     df["total_effect"],
                                     1.0 / df["total_effect"])

    # One row per k: **k is concept_code in (target=Y, code=k)**
    best_per_k = (df.sort_values("effect_strength", ascending=False)
                    .drop_duplicates(subset=["concept_code"], keep="first"))

    HI = hi
    LO = 1.0 / hi

    risk_pool = best_per_k[best_per_k["total_effect"] >= HI].copy()
    prot_pool = best_per_k[best_per_k["total_effect"] <= LO].copy()

    want_risk = top_k // 2
    want_prot = top_k - want_risk

    sel_risk = risk_pool.nlargest(min(len(risk_pool), want_risk), "effect_strength")
    sel_prot = prot_pool.nlargest(min(len(prot_pool), want_prot), "effect_strength")

    selected = pd.concat([sel_risk, sel_prot], ignore_index=True)

    print(f"[SELECT] total unique k={best_per_k['concept_code'].nunique():,}  "
          f"risk={len(risk_pool):,}  prot={len(prot_pool):,}  "
          f"selected(total)={selected['concept_code'].nunique()}  "
          f"with: risk={len(sel_risk)}, prot={len(sel_prot)}")

    return selected.reset_index(drop=True)


def _fetch_k_to_T(conn, outcome_code: str) -> pd.DataFrame:
    """
    Fetch all rows whose LEFT (target) == outcome Y.
    These rows provide the single (Y, k) lines needed to compute T_{kY}.
    """
    q = """
      SELECT m.*,
             tcn.concept_code AS target_concept_code,   -- LEFT  (Y)
             ccn.concept_code AS concept_code           -- RIGHT (k)
      FROM magi_counts_top500 m
      JOIN concept_names tcn ON m.target_concept_code_int = tcn.concept_code_int
      JOIN concept_names ccn ON m.concept_code_int        = ccn.concept_code_int
      WHERE tcn.concept_code = ?                         -- <<< filter on LEFT == Y
    """
    return pd.read_sql_query(q, conn, params=[outcome_code])


def _fetch_subgraph_by_targets(conn, events_list):
    """
    Fetch the induced subgraph for the event set on the LEFT side.
    This captures all rows (X, k) where X ∈ events_set and k ∈ events_set,
    which includes:
      • (Y, k) rows (needed for T_{kY})
      • (j, k) rows (needed for λ_{k,j})
    """
    ph = ",".join(["?"] * len(events_list))
    q = f"""
      SELECT m.*,
             tcn.concept_code AS target_concept_code,   -- LEFT
             ccn.concept_code AS concept_code           -- RIGHT
      FROM magi_counts_top500 m
      JOIN concept_names tcn ON m.target_concept_code_int = tcn.concept_code_int
      JOIN concept_names ccn ON m.concept_code_int        = ccn.concept_code_int
      WHERE tcn.concept_code IN ({ph})
    """
    return pd.read_sql_query(q, conn, params=list(events_list))

# ========= main loop =========
uri = f"file:{MAGI_DB_PATH}?mode=ro"
with sqlite3.connect(uri, uri=True) as conn:
    for T in TARGETS:
        print("\n" + "=" * 100)
        print(f"[RUN] Target = {T}")

        # 1) Y→k rows (target == T) – used for T_{kY}
        k_to_T = _fetch_k_to_T(conn, T)

        # --- NEW: drop exact duplicate rows (all columns identical) ---
        before = len(k_to_T)
        k_to_T = k_to_T.drop_duplicates()
        after = len(k_to_T)
        if after < before:
            print(f"[DEDUP] k_to_T exact-row duplicates removed: {before - after}  (kept {after})")

        if k_to_T.empty:
            print(f"[WARN] No predictor→target rows for {T}; skipping.")
            continue

        # 2) derive totals/effects on the k→T list
        k_to_T = _ensure_derived_cols(k_to_T)

        # 3) select predictors: top-K by total_effect (one row per k = concept_code)
        sel_rows = _select_top_by_te_unique_k(k_to_T, top_k=TOP_K)
        if sel_rows.empty:
            print(f"[WARN] No predictors selected for {T}; skipping.")
            continue

        sub_csv = os.path.join(OUT_DIR, f"risk_prot_{T}.csv")
        sel_rows.to_csv(sub_csv, index=False)
        print(f"[SAVED] Factors → {sub_csv}")

        # IMPORTANT: predictors are the **RIGHT** side (concept_code) in (target=Y, code=k)
        selected_k = set(sel_rows["concept_code"].astype(str))
        print(f"[SELECT] unique k available={k_to_T['concept_code'].nunique():,}  "
            f"selected={len(selected_k):,}")

        # 4) build subgraph among {T} ∪ selected_k
        events_set = selected_k | {T}
        df_trim = _fetch_subgraph_by_targets(conn, sorted(events_set))
        df_trim = df_trim[
            df_trim["target_concept_code"].isin(events_set) &
            df_trim["concept_code"].isin(events_set)
        ].copy()

        # --- NEW: drop exact duplicate rows (all columns identical) ---
        before = len(df_trim)
        df_trim = df_trim.drop_duplicates()
        after = len(df_trim)
        if after < before:
            print(f"[DEDUP] subgraph exact-row duplicates removed: {before - after}  (kept {after})")

        # Recompute derived cols (incl. total_effect) on the subgraph
        df_trim = _ensure_derived_cols(df_trim)

        # Correct orientation in the prints:
        #   k→T rows: concept_code == T
        #   T→j rows: target_concept_code == T
        print(
            f"[TRIM] rows={len(df_trim):,}  events={len(events_set)}  "
            f"Y→k rows (target=={T}) = {(df_trim['target_concept_code'] == T).sum()}  "  # was concept_code==T
            f"j→k rows (target in events_set, ≠{T}) = "
            f"{(df_trim['target_concept_code'].isin(events_set) & (df_trim['target_concept_code'] != T)).sum()}"
        )


        # 5) save subgraph (for audit)
        sub_csv = os.path.join(OUT_DIR, f"magi_subgraph_{T}.csv")
        df_trim.to_csv(sub_csv, index=False)
        print(f"[SAVED] Subgraph → {sub_csv}")

        # 6) run MAGI
        try:
            res = analyze_causal_sequence_py(df_trim, events=None, name_map=None, force_outcome=T)
        except TypeError:
            res = analyze_causal_sequence_py(df_trim)
            outcome_used = res.get("order_used", [None])[-1]
            if outcome_used != T:
                print(f"[NOTE] outcome auto-inferred as {outcome_used}, not {T}")

        # 7) save NON-ZERO coefficients only
        outcome_used = res.get("order_used", [T])[-1]
        coef_df = res["coef_df"].copy()

        # robust: accept 'coef' or 'beta' column name
        coef_col = "coef" if "coef" in coef_df.columns else ("beta" if "beta" in coef_df.columns else None)
        if coef_col is None:
            raise KeyError("Coefficient column not found in coef_df (expected 'coef' or 'beta').")

        # filter to non-zero (tolerance to avoid tiny numerical noise)
        eps = 1e-12
        mask_nz = coef_df[coef_col].astype(float).abs() > eps
        coef_nz = coef_df.loc[mask_nz].copy()

        coef_csv = os.path.join(OUT_DIR, f"magi_coef_{outcome_used}_nonzero.csv")
        coef_nz.to_csv(coef_csv, index=False)

        print(f"[SAVED] Coefficients (non-zero only) → {coef_csv}  "
              f"| kept={len(coef_nz):,} of {len(coef_df):,}  "
              f"| antecedents={len(res.get('order_used', [])) - 1}  "
              f"| used_total_effect={res.get('used_total_effect', True)}")


[RUN] Target = aa_meas_citalopram_rem
[SELECT] total unique k=4,148  risk=3,615  prot=66  selected(total)=1016  with: risk=950, prot=66
[SAVED] Factors → ./MAGI_LASSO/risk_prot_aa_meas_citalopram_rem.csv
[SELECT] unique k available=4,148  selected=1,016


DatabaseError: Execution failed on sql '
      SELECT m.*,
             tcn.concept_code AS target_concept_code,   -- LEFT
             ccn.concept_code AS concept_code           -- RIGHT
      FROM magi_counts_top500 m
      JOIN concept_names tcn ON m.target_concept_code_int = tcn.concept_code_int
      JOIN concept_names ccn ON m.concept_code_int        = ccn.concept_code_int
      WHERE tcn.concept_code IN (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?

In [ ]:

if __name__ == "__main__":
    # 1) Read the counts table
    # Use a raw string for backslashes on Windows
    df = pd.read_csv(os.path.join(f"output.csv"))

    # 2) Map raw codes -> friendly labels
    name_map = {
        "px_CPT4_0025T":      "Cornea",
        "dx_SNOMED_47639008": "Cyst",
        "px_CPT4_00840":      "Anesthesia",
        "aa_meas_citalopram": "Response",
    }

    # 3) Events to keep (AFTER recoding)
    events = ["Cornea", "Cyst", "Anesthesia", "Response"]

    # 4) Run the analysis with different L thresholds
    for L in (30, 15, 10):
        print("\n" + "="*80)
        print(f"[RUN] lambda_min_count = {L}")
        res = analyze_causal_sequence_py(
            df, name_map=name_map, events=events, force_outcome=None, lambda_min_count=L
        )
        print("[ORDER]", " -> ".join(res["temporal_order"]))
        print("[T]");   print(res["T_val"].round(6))
        print("[D]");   print(res["D_val"].round(6))
        print("[β]");   print(res["coef_df"])

        # Example: inspect λ for one antecedent if present
        if len(res["order_used"]) >= 2:
            k0 = res["order_used"][0]
            lam0 = res["lambda_l"].get(k0)
            if lam0 is not None and len(lam0) > 0:
                print(f"[λ for {k0}]")
                print(lam0.round(6))

    # 5) Example: compute probabilities for a simple feature vector
    #    (set Cornea=1, Cyst=0, Anesthesia=1 for demonstration)
    demo = {"Cornea": 1, "Cyst": 1, "Anesthesia": 1}
    res_demo = analyze_causal_sequence_py(
        df, name_map=name_map, events=events, lambda_min_count=30
    )
    p = res_demo["predict_proba"](demo)
    print("\n[DEMO predict_proba] with", demo, "→ P(Response=1) =", round(float(p), 6))

In [ ]:



############### MAGI FUNCTION ###############
"""
MAGI: Dependent Bayes with Temporal Ordering — Reference Implementation
-------------------------------------------------------------------------------
This file adds **step-by-step comments** to the function `analyze_causal_sequence_py`. The comments mirror the proposed
algorithm sections:

1) Determination of Temporal Order
2) Estimation of Dependent Bayes (T-values, λ-links, and D-values)
3) Logistic link to produce P(Y=1 | Z)

INPUT TABLE EXPECTATIONS (long format, one row per directed pair):
- target_concept_code : str (# left node in the edge; in many places this
denotes the earlier event)
- concept_code : str (# right node in the edge; the later event)
- n_code_target : float/int (# co-occurrence count for concept & target
in the *target=Y* stratum; used as 'a')
- n_code_no_target : float/int (# co-occurrence count for concept & target
in the *target=¬Y* stratum; used as 'b')
- n_target : float/int (# total Y count for this target concept, per edge row; we take max within a block)
- n_no_target : float/int (# total ¬Y count; max within a block)
- no_code_no_target : float/int (# optional, computed if missing as n_no_target - n_code_no_target; clipped ≥0)
- n_target_before_code: float/int (# count where target occurred before code)
- n_code_before_target: float/int (# count where code occurred before target)

NOTES ON TEMPORAL COUNTS:
For a row with target_concept_code = A and concept_code = B, the columns
`n_code_before_target` and `n_target_before_code` are interpreted as:
- n_code_before_target: # of persons where **B happened before A**
- n_target_before_code: # of persons where **A happened before B**
We aggregate these across j≠i to compute *temporal scores* for each event.

PIECEWISE, SAMPLE-SIZE–ANCHORED ADJUSTMENTS:
- For odds terms that would be 0 or ∞ due to zero cells, we replace the
offending odds with 1/(N+1) or (N+1)/1, where N is the size of the
appropriate stratum, so all ratios remain finite and interpretable.

RETURN VALUE:
A dict with temporal ordering, T-values, λ-vectors, D-values, coefficients
for a logistic link, a `predict_proba` callable, and trace tables.
"""

def analyze_causal_sequence_py(
    data: Union[str, pd.DataFrame],
    name_map: Dict[str, str],
    events: List[str],
    force_outcome=None,
) -> Dict[str, Any]:
    """Compute temporal order, dependent-Bayes direct effects (D), and
    a logistic-link probability for outcome Y from pairwise counts.

    Parameters
    ----------
    data : str | DataFrame
        CSV path or in-memory DataFrame with the columns described above.
    name_map : Dict[str, str]
        Optional mapping raw code -> friendly label. If provided, both
        `target_concept_code` and `concept_code` are replaced.
    events : List[str]
        List of event names/codes to restrict the analysis to. If `None`,
        events are auto-detected from the table and intersected.
    force_outcome : str | None
        If provided and found among events, this event is forced to be the
        **final** node (i.e., the outcome) in the temporal order.

    Returns
    -------
    Dict[str, Any]
        - sorted_scores : pd.Series of temporal scores (desc)
        - temporal_order: list of events (outcome at the end)
        - order_used    : same as temporal_order
        - T_val         : pd.Series of total effects T_{k,Y}
        - D_val         : pd.Series of direct effects D_{k,Y}
        - coef_df       : pd.DataFrame of coefficients (β_k and intercept)
        - lambda_l      : dict[str -> pd.Series] of λ_{k,j} vectors
        - trace_df      : pd.DataFrame detailing the backward recursion steps
        - invalid_predictors: list of predictors whose log(D) was invalid
        - beta_0, beta, logit_predictors, predict_proba: logistic elements
    """

    # ---------------------------------------------------------------------
    # 0) INGEST & BASIC VALIDATION
    # ---------------------------------------------------------------------
    if isinstance(data, str):
        # Read from CSV path
        df = pd.read_csv(data)
    else:
        # Work on a copy to avoid mutating caller's object
        df = data.copy()

    # Ensure required identifier columns are present
    for col in ["target_concept_code", "concept_code"]:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # Optional recoding to human-friendly labels
    if name_map:
        df["target_concept_code"] = df["target_concept_code"].replace(name_map)
        df["concept_code"] = df["concept_code"].replace(name_map)

    # Numeric columns the algorithm expects
    need = [
        "n_code_target", "n_code_no_target", "n_target", "n_no_target",
        "n_target_before_code", "n_code_before_target",
    ]
    missing = [c for c in need if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {', '.join(missing)}")

    # If precomputed total_effect exists (for λ or sanity checks), coerce to numeric
    has_total = "total_effect" in df.columns
    if has_total:
        df["total_effect"] = pd.to_numeric(df["total_effect"], errors="coerce")

    # ---------------------------------------------------------------------
    # 1) EVENT SET (optional auto-detect) & TYPE COERCION
    # ---------------------------------------------------------------------
    if events is None:
        # Auto-detect: intersect events appearing on both sides of edges
        ev_targets = df["target_concept_code"].dropna().astype(str).unique().tolist()
        ev_code = df["concept_code"].dropna().astype(str).unique().tolist()
        events = sorted(set(ev_targets).intersection(ev_code))
        if len(events) == 0:
            # Fall back to union if intersection is empty
            events = sorted(set(ev_targets) | set(ev_code))
    if len(events) < 2:
        raise ValueError("Need at least two events after auto-detection.")

    # Keep only rows whose endpoints are both in the chosen event set
    df = df[df["target_concept_code"].isin(events) & df["concept_code"].isin(events)].copy()

    # Coerce numeric columns robustly (invalid -> NaN); subsequent ops handle NaNs
    for c in need:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # If `no_code_no_target` missing, derive as (n_no_target - n_code_no_target) ≥ 0
    if "no_code_no_target" not in df.columns:
        df["no_code_no_target"] = (df["n_no_target"] - df["n_code_no_target"]).clip(lower=0)
    else:
        df["no_code_no_target"] = pd.to_numeric(df["no_code_no_target"], errors="coerce").clip(lower=0)

    # Helper: total count for an event (max n_target where that event is target)
    # This mirrors your original choice; change to .sum() if warranted.
    def C_of(ev: str) -> float:
        sub = df[df["target_concept_code"] == ev]
        if sub.empty:
            return float("nan")
        C = pd.to_numeric(sub["n_target"], errors="coerce").max()
        return float(C) if pd.notna(C) and np.isfinite(C) else float("nan")

    # Helper: numerically safe log (log(1)=0 for invalid/≤0)
    def safe_log(x: float) -> float:
        try:
            xv = float(x)
        except (TypeError, ValueError):
            return 0.0
        if not np.isfinite(xv) or xv <= 0.0:
            return 0.0
        return math.log(xv)

    # ---------------------------------------------------------------------
    # 2) TEMPORAL ORDER — pairwise-before counts → per-node score
    # ---------------------------------------------------------------------
    # Score(Z_i) = Σ_{j≠i} [ C(Z_i≪Z_j) - C(Z_j≪Z_i) + C(Z_i ∩ ¬Z_j) - C(Z_j ∩ ¬Z_i) ]
    # Here we implement the *before/after* portion using the provided columns.
    # If only presence/absence terms are available, you may approximate using
    # the last two terms.
    scores: Dict[str, float] = {}
    for zk in events:
        s = 0.0
        for zj in [x for x in events if x != zk]:
            # For the pair (zj, zk), we interpret:
            #   n_code_before_target   — # where zk (code) before zj (target)
            #   n_target_before_code   — # where zj (target) before zk (code)
            rowr = df[(df["target_concept_code"] == zj) & (df["concept_code"] == zk)]
            if not rowr.empty:
                s += float(rowr["n_code_before_target"].sum(skipna=True) -
                           rowr["n_target_before_code"].sum(skipna=True))
        scores[zk] = s

    sorted_scores = pd.Series(scores).sort_values(ascending=False)

    # Outcome selection:
    #  - If `force_outcome` is provided and present, put it at the end.
    #  - Else, default to the top-scoring node as outcome.
    if force_outcome and (force_outcome in sorted_scores.index):
        outcome_event = force_outcome
        temporal_order = [ev for ev in sorted_scores.index if ev != outcome_event] + [outcome_event]
    else:
        outcome_event = sorted_scores.index[0]
        temporal_order = [ev for ev in sorted_scores.index if ev != outcome_event] + [outcome_event]

    # Propagation order is the temporal order; last is outcome Y
    events_order = temporal_order
    outcome = events_order[-1]
    antecedents = events_order[:-1]

    # ---------------------------------------------------------------------
    # 3) T-VALUES (TOTAL EFFECTS) & λ-LINKS BETWEEN ANTECEDENTS
    # ---------------------------------------------------------------------
    # For each antecedent k, we compute T_{k,Y} as an odds ratio between
    # strata Z_k=1 and Z_k=0 with sample-size–anchored fixes for zero cells.
    # For λ_{k,j} (dependence of j given k), we first use precomputed
    # `total_effect` if present.
    # If λ is missing (no recorded dependence), assume independence
    # and treat the conditional contribution as 0 in the adjustment sum.

    T_val = pd.Series(0.0, index=antecedents, dtype=float)  # T_{k,Y}
    D_val = pd.Series(np.nan, index=antecedents, dtype=float)  # D_{k,Y}
    lambda_l: Dict[str, pd.Series] = {}  # per-k vector of λ_{k,j} for j after k and before Y

    for k in antecedents:
        # ---- Contingency for (k, outcome) ----
        row_ko = df[(df["target_concept_code"] == k) & (df["concept_code"] == outcome)]

        # a: Y∩k    b: ¬Y∩k    c: Y∩¬k    d: ¬Y∩¬k
        a = float(row_ko["n_code_target"].sum(skipna=True))            # co-occurrence in Y
        b = float(row_ko["n_code_no_target"].sum(skipna=True))         # co-occurrence in ¬Y

        # If n_target_no_code absent, approximate c by (max n_target - a)
        if "n_target_no_code" in row_ko.columns:
            c = float(row_ko["n_target_no_code"].sum(skipna=True))
        else:
            c = float(pd.to_numeric(row_ko["n_target"], errors="coerce").max() - a)

        # If no_code_no_target absent, approximate d by (max n_no_target - b)
        if "no_code_no_target" in row_ko.columns:
            d = float(row_ko["no_code_no_target"].sum(skipna=True))
        else:
            d = float(pd.to_numeric(row_ko["n_no_target"], errors="coerce").max() - b)

        N1, N0 = a + b, c + d  # stratum sizes for Z_k=1 and Z_k=0

        # ---- Sample-size–anchored odds for Z_k=1 ----
        if a == 0:
            odds_pos_adj = 1.0 / (N1 + 1.0)  # 0 → tiny positive odds
        elif b == 0:
            odds_pos_adj = (N1 + 1.0) / 1.0  # ∞ → capped by (N1+1)
        else:
            odds_pos_adj = a / b

        # ---- Sample-size–anchored odds for Z_k=0 ----
        if c == 0:
            odds_neg_adj = 1.0 / (N0 + 1.0)
        elif d == 0:
            odds_neg_adj = (N0 + 1.0) / 1.0
        else:
            odds_neg_adj = c / d

        # Total effect T_{k,Y}
        T_val.loc[k] = float(odds_pos_adj / odds_neg_adj)

        # ---- λ_{k,j}: dependence of j on k for nodes j after k (and before Y) ----
        pos_k = events_order.index(k)
        js = events_order[pos_k + 1 : -1] if pos_k < len(events_order) - 1 else []

        lam_pairs = []
        for j in js:
            row_kj = df[(df["target_concept_code"] == k) & (df["concept_code"] == j)]
            if row_kj.empty:
                lam_pairs.append((j, 0.0))  # no evidence of dependence
                continue

            # Prefer precomputed total_effect if present for this edge
            #te = float(pd.to_numeric(row_kj["total_effect"], errors="coerce").max()) if has_total else float("nan")
            #if np.isfinite(te):
            #    lam_pairs.append((j, te))
            #    continue

            # Otherwise approximate λ with piecewise, size-anchored logic
            # C11 = C(j∩k); Cj_not_k = C(j∩¬k); Ck = C(k)
            C11 = float(row_kj["n_code_target"].sum(skipna=True))  # re-using the same column name for co-occurrence
            if "n_code_no_target" in row_kj.columns:
                Cj_not_k = float(row_kj["n_code_no_target"].sum(skipna=True))
            else:
                Cj = C_of(j)
                Cj_not_k = 0.0 if (not np.isfinite(Cj)) else max(Cj - C11, 0.0)
            Ck = C_of(k)

            if Cj_not_k == 0:
                L = 1.0 + C11          # always-with-k → boost
            elif C11 == 0:
                L = 1.0 / (1.0 + Cj_not_k)  # never-with-k → downweight
            elif np.isfinite(Ck) and Ck > 0:
                L = C11 / Ck           # normalized co-occurrence
            else:
                L = 0.0

            lam_pairs.append((j, float(L)))

        lambda_l[k] = pd.Series({j: v for j, v in lam_pairs}, dtype=float)

    # ---------------------------------------------------------------------
    # 4) BACKWARD RECURSION — resolve D_{k,Y} from T and λ
    # ---------------------------------------------------------------------
    # D_{k,Y} = ( T_{k,Y} - Σ_i λ_{k,k+i} * D_{k+i,Y} ) / ( 1 - Σ_i λ_{k,k+i} )
    # Start at the last antecedent (just before Y): D := T, since there are no
    # downstream nodes to adjust for.

    trace_rows = []  # for human-auditable tracing of the recursion

    last_anc = antecedents[-1] if antecedents else None
    if last_anc is not None:
        D_val.loc[last_anc] = T_val.loc[last_anc]
        trace_rows.append({
            "stage": "Last 2 Nodes",
            "nodes": f"{last_anc} - {outcome}",
            "k": last_anc,
            "T_kY": T_val.loc[last_anc],
            "lambda_terms": None,
            "sum_lambda": 0.0,
            "D_kY": D_val.loc[last_anc],
            "log_D": safe_log(D_val.loc[last_anc]),
        })

    if len(antecedents) > 1:
        # Walk backwards through the remaining antecedents
        for k in list(reversed(antecedents))[1:]:
            lam_vec = lambda_l.get(k, pd.Series(dtype=float))
            # When computing the adjustment, use **only** D-values for nodes that
            # are after k and already resolved (present in lam_vec index).
            code = list(lam_vec.index)
            num = T_val.loc[k] - float(np.nansum(lam_vec.reindex(code).values * D_val.reindex(code).values))
            den = 1.0 - float(np.nansum(lam_vec.values))  # may approach 0 if λ's are large

            # If den is pathological (≤0 or NaN), fall back to T (neutralization).
            D_val.loc[k] = (num / den) if np.isfinite(num / den) else T_val.loc[k]

            span = len(events_order) - events_order.index(k) + 1
            lam_str = ", ".join(
                f"λ_{events_order.index(k)+1}{events_order.index(c)+1}={lam_vec[c]:.6f}"
                for c in code
            ) if len(lam_vec) else None

            trace_rows.append({
                "stage": f"Last {span} Nodes",
                "nodes": " - ".join([k] + events_order[events_order.index(k)+1:]),
                "k": k,
                "T_kY": T_val.loc[k],
                "lambda_terms": lam_str,
                "sum_lambda": float(np.nansum(lam_vec.values)),
                "D_kY": D_val.loc[k],
                "log_D": safe_log(D_val.loc[k]),
            })

    trace_df = pd.DataFrame(trace_rows)

    # ---------------------------------------------------------------------
    # 5) COEFFICIENTS — map D's onto a logistic link
    # ---------------------------------------------------------------------
    # We model:  logit P(Y=1 | Z) = β0 + Σ_k β_k * Z_k,  with β_k = log D_{k,Y}
    # Intercept β0 is set by the marginal prevalence of Y (from `n_target` &
    # `n_no_target`) for the outcome rows.

    resp_rows = df[df["target_concept_code"] == outcome]
    if resp_rows.empty:
        raise ValueError(f"No rows for outcome '{outcome}'.")

    n_t = resp_rows["n_target"].dropna().iloc[0] if resp_rows["n_target"].dropna().size else np.nan
    n_n = resp_rows["n_no_target"].dropna().iloc[0] if resp_rows["n_no_target"].dropna().size else np.nan
    denom = n_t + n_n
    p_y = 0.5 if (not np.isfinite(denom) or denom <= 0) else (n_t / denom)
    beta_0 = float(np.log(p_y / (1 - p_y)))  # stable enough for p∈(0,1)

    # β_k = log(D_{k,Y}); protect against non-positive D by mapping to 0
    D_clean = pd.to_numeric(D_val, errors="coerce").astype(float)
    D_pos = D_clean.where(D_clean > 0)

    with np.errstate(divide="ignore", invalid="ignore"):
        beta_vals = np.log(D_pos.to_numpy())
    beta_k_raw = pd.Series(beta_vals, index=D_val.index)
    invalid_predictors = list(beta_k_raw[~np.isfinite(beta_k_raw)].index)

    beta_k = beta_k_raw.copy()
    beta_k[~np.isfinite(beta_k)] = 0.0  # neutralize invalid predictors

    coef_df = pd.DataFrame({
        "predictor": list(beta_k.index) + ["(intercept)"],
        "beta": list(beta_k.astype(float).values) + [beta_0],
    })

    # ---------------------------------------------------------------------
    # 6) PREDICT_PROBA — vectorized logistic link
    # ---------------------------------------------------------------------
    predictors = list(beta_k.index)
    beta_vec = beta_k.astype(float).values

    def predict_proba(z: Union[Dict[str, Any], pd.Series, np.ndarray, List[float], pd.DataFrame]) -> Union[float, np.ndarray, pd.Series]:
        """Compute P(Y=1 | Z) using the logistic link.

        Accepts:
          - dict/Series mapping predictor name -> 0/1
          - 1D/2D numpy/list with columns ordered as `predictors`
          - DataFrame with columns containing any/all of `predictors` (others ignored)

        Returns:
          - float for 1D inputs; np.ndarray or pd.Series for vectorized inputs
        """
        if isinstance(z, pd.DataFrame):
            Z = z.reindex(columns=predictors, fill_value=0).astype(float).to_numpy()
            eta = beta_0 + Z @ beta_vec
            # Stable sigmoid via clipping; avoids overflow for extreme η
            return 1.0 / (1.0 + np.exp(-np.clip(eta, -700, 700)))

        if isinstance(z, (dict, pd.Series)):
            v = np.array([float(z.get(p, 0.0)) for p in predictors], dtype=float)
            eta = beta_0 + float(v @ beta_vec)
            return float(1.0 / (1.0 + np.exp(-np.clip(eta, -700, 700))))

        arr = np.asarray(z, dtype=float)
        if arr.ndim == 1:
            if arr.size != len(predictors):
                raise ValueError(f"Expected {len(predictors)} features in order: {predictors}")
            eta = beta_0 + float(arr @ beta_vec)
            return float(1.0 / (1.0 + np.exp(-np.clip(eta, -700, 700))))
        else:
            if arr.shape[1] != len(predictors):
                raise ValueError(f"Expected shape (*, {len(predictors)}), got {arr.shape}")
            eta = beta_0 + arr @ beta_vec
            return 1.0 / (1.0 + np.exp(-np.clip(eta, -700, 700)))

    # ---------------------------------------------------------------------
    # 7) PACKAGE RESULTS
    # ---------------------------------------------------------------------
    return {
        "sorted_scores": sorted_scores,
        "temporal_order": temporal_order,
        "order_used": events_order,
        "T_val": T_val,
        "D_val": D_val,
        "coef_df": coef_df,
        "lambda_l": lambda_l,
        "trace_df": trace_df,
        "invalid_predictors": invalid_predictors,
        # Logistic link outputs:
        "beta_0": beta_0,
        "beta": pd.Series(beta_vec, index=predictors, dtype=float),
        "logit_predictors": predictors,
        "predict_proba": predict_proba,
    }

# ========= helpers =========

def _ensure_derived_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ensure derived counts (n_target_no_code, no_code_no_target) exist,
    coerce numeric columns safely, and (if absent) compute per-edge total_effect
    using sample-size–anchored odds. Returns a copy.
    """
    out = df.copy()

    # ---- 1) Base numeric columns ----
    base = ["n_code_target", "n_code_no_target", "n_target", "n_no_target"]
    for c in base:
        if c not in out.columns:
            out[c] = 0
        out[c] = pd.to_numeric(out[c], errors="coerce").fillna(0).clip(lower=0).astype(float)

    # ---- 2) Complements (clip to ≥ 0) ----
    # Y & (k=0)
    if "n_target_no_code" not in out.columns:
        out["n_target_no_code"] = (out["n_target"] - out["n_code_target"]).clip(lower=0)
    else:
        out["n_target_no_code"] = pd.to_numeric(out["n_target_no_code"], errors="coerce").fillna(0).clip(lower=0)

    # (k=0) & (Y=0)
    if "no_code_no_target" not in out.columns:
        out["no_code_no_target"] = (out["n_no_target"] - out["n_code_no_target"]).clip(lower=0)
    else:
        out["no_code_no_target"] = pd.to_numeric(out["no_code_no_target"], errors="coerce").fillna(0).clip(lower=0)

    # ---- 3) total_effect (odds_pos / odds_neg) if absent ----
    if "total_effect" not in out.columns:
        a  = out["n_code_target"].astype(float)        # k=1, Y=1
        b  = out["n_code_no_target"].astype(float)     # k=1, Y=0
        c  = out["n_target_no_code"].astype(float)     # k=0, Y=1
        d  = out["no_code_no_target"].astype(float)    # k=0, Y=0

        N1 = a + b
        N0 = c + d

        # Sample-size–anchored odds with neutral fallback when stratum size is zero
        with np.errstate(divide="ignore", invalid="ignore"):
            odds_pos = np.where(
                N1 == 0, 1.0,                              # no data in Z_k=1 → neutral
                np.where(
                    (a > 0) & (b > 0), a / b,
                    np.where(b == 0, N1 + 1.0, 1.0 / (N1 + 1.0))
                )
            )
            odds_neg = np.where(
                N0 == 0, 1.0,                              # no data in Z_k=0 → neutral
                np.where(
                    (c > 0) & (d > 0), c / d,
                    np.where(d == 0, N0 + 1.0, 1.0 / (N0 + 1.0))
                )
            )

            te = odds_pos / odds_neg

        # Clean any residual non-finite values to neutral 1.0
        te = np.where(np.isfinite(te), te, 1.0)
        out["total_effect"] = te.astype(float)

    return out

# TOPK_TE
def _select_top_by_te_unique_k(k_to_T: pd.DataFrame, top_k: int) -> pd.DataFrame:
    """
    Select predictors strictly by total_effect (descending), keeping ONE row per predictor k.
    Predictor k is the LEFT side in the row → column 'target_concept_code'.
    """
    df = k_to_T.copy()
    df["total_effect"] = pd.to_numeric(df["total_effect"], errors="coerce")
    df = df.dropna(subset=["total_effect"])
    if df.empty:
        print("[SELECT] no rows with total_effect; returning empty selection.")
        return df

    # Sort by TE desc, then tie-break deterministically by k code
    sort_cols = ["total_effect", "target_concept_code"]
    df = df.sort_values(sort_cols, ascending=[False, True])

    # Keep most-extreme row per k (target_concept_code)
    best_per_k = df.drop_duplicates(subset=["target_concept_code"], keep="first").reset_index(drop=True)

    take_n = min(int(top_k), len(best_per_k))
    selected = best_per_k.head(take_n).copy()
    selected.insert(0, "rank_by_TE", np.arange(1, len(selected)+1))
    return selected

# TOPK_HI/LO
def _select_top_by_te_unique_k(k_to_T: pd.DataFrame, top_k: int = TOP_K,
                                   hi: float = 1.5) -> pd.DataFrame:
    """
    Balanced selection without fallback:
      - Take up to top_k//2 strongest risk (TE >= hi)
      - Take up to top_k - top_k//2 strongest protective (TE <= 1/hi)
      - If either side is short, do NOT fill from elsewhere; total may be < top_k
      - Return one row per k (target_concept_code), ranked by extremeness
    """
    df = k_to_T.copy()

    # Ensure total_effect numeric
    df["total_effect"] = pd.to_numeric(df["total_effect"], errors="coerce")
    df = df.dropna(subset=["total_effect"]).copy()
    if df.empty:
        print("[SELECT] no rows with total_effect; returning empty selection.")
        return df

    # Extremeness symmetrical around 1
    df["effect_strength"] = np.where(df["total_effect"] >= 1.0,
                                     df["total_effect"],
                                     1.0 / df["total_effect"])

    # One row per k (keep most extreme)
    best_per_k = (df.sort_values("effect_strength", ascending=False)
                    .drop_duplicates(subset=["target_concept_code"], keep="first"))

    HI = hi
    LO = 1.0 / hi

    risk_pool = best_per_k[best_per_k["total_effect"] >= HI].copy()
    prot_pool = best_per_k[best_per_k["total_effect"] <= LO].copy()

    # Target split
    want_risk = top_k // 2
    want_prot = top_k - want_risk

    # Take strongest from each side, capped by availability
    take_risk = min(len(risk_pool), want_risk)
    take_prot = min(len(prot_pool), want_prot)

    sel_risk = risk_pool.nlargest(take_risk, "effect_strength")
    sel_prot = prot_pool.nlargest(take_prot, "effect_strength")

    selected = pd.concat([sel_risk, sel_prot], ignore_index=True)

    print(f"[SELECT] total unique k={len(best_per_k):,}  "
          f"risk={len(risk_pool):,}  prot={len(prot_pool):,}  "
          f"selected(total)={selected['target_concept_code'].nunique()}  "
          f"with: risk={take_risk}, prot={take_prot}")

    return selected.reset_index(drop=True)
def _fetch_k_to_T(conn, target_code: str) -> pd.DataFrame:
    """Fetch only k→T rows (concept_code == target)."""
    q = """
      SELECT m.*,
             tcn.concept_code AS target_concept_code,   -- k (predictor)
             ccn.concept_code AS concept_code           -- T (outcome)
      FROM magi_counts_top500 m
      JOIN concept_names tcn ON m.target_concept_code_int = tcn.concept_code_int
      JOIN concept_names ccn ON m.concept_code_int        = ccn.concept_code_int
      WHERE ccn.concept_code = ?
    """
    return pd.read_sql_query(q, conn, params=[target_code])

def _fetch_subgraph_by_targets(conn, events_list):
    """Fetch edges with target_concept_code IN events_set (single IN to avoid 999 param issues)."""
    ph = ",".join(["?"] * len(events_list))
    q = f"""
      SELECT m.*,
             tcn.concept_code AS target_concept_code,
             ccn.concept_code AS concept_code
      FROM magi_counts_top500 m
      JOIN concept_names tcn ON m.target_concept_code_int = tcn.concept_code_int
      JOIN concept_names ccn ON m.concept_code_int        = ccn.concept_code_int
      WHERE tcn.concept_code IN ({ph})
    """
    return pd.read_sql_query(q, conn, params=list(events_list))
def _fetch_k_to_T(conn, target_code: str) -> pd.DataFrame:
    """Fetch only k→T rows (concept_code == target)."""
    q = """
      SELECT m.*,
             tcn.concept_code AS target_concept_code,   -- k (predictor)
             ccn.concept_code AS concept_code           -- T (outcome)
      FROM magi_counts_top500 m
      JOIN concept_names tcn ON m.target_concept_code_int = tcn.concept_code_int
      JOIN concept_names ccn ON m.concept_code_int        = ccn.concept_code_int
      WHERE ccn.concept_code = ?
    """
    return pd.read_sql_query(q, conn, params=[target_code])

def _fetch_subgraph_by_targets(conn, events_list, counts_table="magi_counts_top500"):
    """
    Return rows whose BOTH ends are in events_list by joining against a TEMP table,
    avoiding SQLite's 'too many SQL variables' limit.
    """
    # normalize/guard
    ev = sorted({str(x) for x in events_list if x is not None})
    if not ev:
        return pd.DataFrame()

    # create temp table of events
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS tmp_events")
    cur.execute("CREATE TEMP TABLE tmp_events (concept_code TEXT PRIMARY KEY)")
    cur.executemany("INSERT OR IGNORE INTO tmp_events(concept_code) VALUES (?)",
                    [(x,) for x in ev])
    conn.commit()  # ensure visibility to read_sql_query

    q = f"""
        SELECT
            m.*,
            tcn.concept_code AS target_concept_code,
            ccn.concept_code AS concept_code
        FROM {counts_table} AS m
        JOIN concept_names AS tcn ON m.target_concept_code_int = tcn.concept_code_int
        JOIN concept_names AS ccn ON m.concept_code_int        = ccn.concept_code_int
        JOIN tmp_events AS te1     ON tcn.concept_code = te1.concept_code   -- target end in set
        JOIN tmp_events AS te2     ON ccn.concept_code = te2.concept_code   -- concept end in set
    """
    df = pd.read_sql_query(q, conn)  # no giant params list needed
    # optional cleanup
    cur.execute("DROP TABLE IF EXISTS tmp_events")
    return df

## Remove dup
# ========= main loop =========
uri = f"file:{MAGI_DB_PATH}?mode=ro"
with sqlite3.connect(uri, uri=True) as conn:
    for T in TARGETS:
        print("\n" + "=" * 100)
        print(f"[RUN] Target = {T}")

        # 1) k→T rows (concept_code == T)
        k_to_T = _fetch_k_to_T(conn, T)

        # --- NEW: drop exact duplicate rows (all columns identical) ---
        before = len(k_to_T)
        k_to_T = k_to_T.drop_duplicates()
        after = len(k_to_T)
        if after < before:
            print(f"[DEDUP] k_to_T exact-row duplicates removed: {before - after}  (kept {after})")

        if k_to_T.empty:
            print(f"[WARN] No predictor→target rows for {T}; skipping.")
            continue

        # 2) derive totals/effects on the k→T list
        k_to_T = _ensure_derived_cols(k_to_T)

        # 3) select predictors: top-K by total_effect (one row per k)
        sel_rows = _select_top_by_te_unique_k(k_to_T, top_k=TOP_K)
        if sel_rows.empty:
            print(f"[WARN] No predictors selected for {T}; skipping.")
            continue

        sub_csv = os.path.join(OUT_DIR, f"risk_prot_{T}.csv")
        df_trim.to_csv(sub_csv, index=False)
        print(f"[SAVED] Factors → {sub_csv}")

        # IMPORTANT: predictors are the LEFT side (target_concept_code)
        selected_k = set(sel_rows["target_concept_code"].astype(str))
        print(f"[SELECT] unique k available={k_to_T['target_concept_code'].nunique():,}  "
              f"selected={len(selected_k):,}")

        # 4) build subgraph among {T} ∪ selected_k
        events_set = selected_k | {T}
        df_trim = _fetch_subgraph_by_targets(conn, sorted(events_set))
        df_trim = df_trim[
            df_trim["target_concept_code"].isin(events_set) &
            df_trim["concept_code"].isin(events_set)
        ].copy()

        # --- NEW: drop exact duplicate rows (all columns identical) ---
        before = len(df_trim)
        df_trim = df_trim.drop_duplicates()
        after = len(df_trim)
        if after < before:
            print(f"[DEDUP] subgraph exact-row duplicates removed: {before - after}  (kept {after})")

        # Recompute derived cols (incl. total_effect) on the subgraph
        df_trim = _ensure_derived_cols(df_trim)

        # Correct orientation in the prints:
        #   k→T rows: concept_code == T
        #   T→j rows: target_concept_code == T
        print(f"[TRIM] rows={len(df_trim):,}  events={len(events_set)}  "
              f"k→T rows={(df_trim['concept_code'] == T).sum()}  "
              f"T→j rows={(df_trim['target_concept_code'] == T).sum()}")

        # 5) save subgraph (for audit)
        sub_csv = os.path.join(OUT_DIR, f"magi_subgraph_{T}.csv")
        df_trim.to_csv(sub_csv, index=False)
        print(f"[SAVED] Subgraph → {sub_csv}")

        # 6) run MAGI
        try:
            res = analyze_causal_sequence_py(df_trim, events=None, name_map=None, force_outcome=T)
        except TypeError:
            res = analyze_causal_sequence_py(df_trim)
            outcome_used = res.get("order_used", [None])[-1]
            if outcome_used != T:
                print(f"[NOTE] outcome auto-inferred as {outcome_used}, not {T}")

        # 7) save NON-ZERO coefficients only
        outcome_used = res.get("order_used", [T])[-1]
        coef_df = res["coef_df"].copy()

        # robust: accept 'coef' or 'beta' column name
        coef_col = "coef" if "coef" in coef_df.columns else ("beta" if "beta" in coef_df.columns else None)
        if coef_col is None:
            raise KeyError("Coefficient column not found in coef_df (expected 'coef' or 'beta').")

        # filter to non-zero (tolerance to avoid tiny numerical noise)
        eps = 1e-12
        mask_nz = coef_df[coef_col].astype(float).abs() > eps
        coef_nz = coef_df.loc[mask_nz].copy()

        coef_csv = os.path.join(OUT_DIR, f"magi_coef_{outcome_used}_nonzero.csv")
        coef_nz.to_csv(coef_csv, index=False)

        print(f"[SAVED] Coefficients (non-zero only) → {coef_csv}  "
              f"| kept={len(coef_nz):,} of {len(coef_df):,}  "
              f"| antecedents={len(res.get('order_used', [])) - 1}  "
              f"| used_total_effect={res.get('used_total_effect', True)}")
# ========= main loop =========
import os, sqlite3

# ensure OUT_DIR exists
os.makedirs(OUT_DIR, exist_ok=True)

uri = f"file:{MAGI_DB_PATH}?mode=ro"
with sqlite3.connect(uri, uri=True) as conn:
    for T in TARGETS:
        print("\n" + "=" * 100)
        print(f"[RUN] Target = {T}")

        # 1) k→T rows (concept_code == T)
        k_to_T = _fetch_k_to_T(conn, T)

        # --- NEW: drop exact duplicate rows (all columns identical) ---
        before = len(k_to_T)
        k_to_T = k_to_T.drop_duplicates()
        after = len(k_to_T)
        if after < before:
            print(f"[DEDUP] k_to_T exact-row duplicates removed: {before - after}  (kept {after})")

        if k_to_T.empty:
            print(f"[WARN] No predictor→target rows for {T}; skipping.")
            continue

        # 2) derive totals/effects on the k→T list
        k_to_T = _ensure_derived_cols(k_to_T)

        # 3) select predictors: top-K by total_effect (one row per k)
        sel_rows = _select_top_by_te_unique_k(k_to_T, top_k=TOP_K)
        if sel_rows.empty:
            print(f"[WARN] No predictors selected for {T}; skipping.")
            continue

        # save the selected k→T factor list (NOT df_trim; it doesn't exist yet)
        factors_csv = os.path.join(OUT_DIR, f"risk_prot_{T}.csv")
        sel_rows.to_csv(factors_csv, index=False)
        print(f"[SAVED] Factors → {factors_csv}")

        # IMPORTANT: predictors are the LEFT side (target_concept_code)
        selected_k = set(sel_rows["target_concept_code"].astype(str))
        print(
            f"[SELECT] unique k available={k_to_T['target_concept_code'].nunique():,}  "
            f"selected={len(selected_k):,}"
        )

        # 4) build subgraph among {T} ∪ selected_k
        events_set = selected_k | {str(T)}
        df_trim = _fetch_subgraph_by_targets(conn, sorted(events_set))
        # keep only rows fully inside the event set
        df_trim = df_trim[
            df_trim["target_concept_code"].astype(str).isin(events_set) &
            df_trim["concept_code"].astype(str).isin(events_set)
        ].copy()

        # --- NEW: drop exact duplicate rows (all columns identical) ---
        before = len(df_trim)
        df_trim = df_trim.drop_duplicates()
        after = len(df_trim)
        if after < before:
            print(f"[DEDUP] subgraph exact-row duplicates removed: {before - after}  (kept {after})")

        # Recompute derived cols (incl. total_effect) on the subgraph
        df_trim = _ensure_derived_cols(df_trim)

        # Correct orientation in the prints:
        #   k→T rows: concept_code == T
        #   T→j rows: target_concept_code == T
        n_k_to_T = int((df_trim["concept_code"].astype(str) == str(T)).sum())
        n_T_to_j = int((df_trim["target_concept_code"].astype(str) == str(T)).sum())
        print(
            f"[TRIM] rows={len(df_trim):,}  events={len(events_set)}  "
            f"k→T rows={n_k_to_T}  T→j rows={n_T_to_j}"
        )

        # 5) save subgraph (for audit)
        sub_csv = os.path.join(OUT_DIR, f"magi_subgraph_{T}.csv")
        df_trim.to_csv(sub_csv, index=False)
        print(f"[SAVED] Subgraph → {sub_csv}")

        # 6) run MAGI
        try:
            res = analyze_causal_sequence_py(df_trim, events=None, name_map=None, force_outcome=str(T))
        except TypeError:
            # fallback for older signature
            res = analyze_causal_sequence_py(df_trim)
            outcome_used_fallback = res.get("order_used", [None])[-1]
            if outcome_used_fallback != str(T):
                print(f"[NOTE] outcome auto-inferred as {outcome_used_fallback}, not {T}")

        if not isinstance(res, dict) or "coef_df" not in res:
            raise RuntimeError("analyze_causal_sequence_py did not return expected result (missing 'coef_df').")

        # 7) save NON-ZERO coefficients only
        outcome_used = res.get("order_used", [str(T)])[-1]
        coef_df = res["coef_df"].copy()

        # robust: accept 'coef' or 'beta' column name
        coef_col = "coef" if "coef" in coef_df.columns else ("beta" if "beta" in coef_df.columns else None)
        if coef_col is None:
            raise KeyError("Coefficient column not found in coef_df (expected 'coef' or 'beta').")

        # filter to non-zero (tolerance to avoid tiny numerical noise)
        eps = 1e-12
        mask_nz = coef_df[coef_col].astype(float).abs() > eps
        coef_nz = coef_df.loc[mask_nz].copy()

        coef_csv = os.path.join(OUT_DIR, f"magi_coef_{outcome_used}_nonzero.csv")
        coef_nz.to_csv(coef_csv, index=False)

        print(
            f"[SAVED] Coefficients (non-zero only) → {coef_csv}  "
            f"| kept={len(coef_nz):,} of {len(coef_df):,}  "
            f"| antecedents={max(0, len(res.get('order_used', [])) - 1)}  "
            f"| used_total_effect={res.get('used_total_effect', True)}"
        )

# ========= main loop (remove duplicates; keep ALL predictors) =========
uri = f"file:{MAGI_DB_PATH}?mode=ro"
with sqlite3.connect(uri, uri=True) as conn:
    for T in TARGETS:
        print("\n" + "=" * 100)
        print(f"[RUN] Target = {T}")

        # 1) k→T rows (concept_code == T)
        k_to_T = _fetch_k_to_T(conn, T)

        # --- drop exact duplicate rows (all columns identical) ---
        before = len(k_to_T)
        k_to_T = k_to_T.drop_duplicates()
        after = len(k_to_T)
        if after < before:
            print(f"[DEDUP] k_to_T exact-row duplicates removed: {before - after}  (kept {after})")

        if k_to_T.empty:
            print(f"[WARN] No predictor→target rows for {T}; skipping.")
            continue

        # 2) derive totals/effects on the k→T list
        k_to_T = _ensure_derived_cols(k_to_T)

        # 3) KEEP ALL predictors (no TOP-K filtering)
        #    'selected_k' is the full unique set of antecedent codes on the LEFT side (target_concept_code)
        selected_k = set(k_to_T["target_concept_code"].astype(str).unique())
        print(f"[SELECT] unique k available={len(selected_k):,}  (keeping ALL)")

        # Save the full k→T table for audit/import
        vars_csv = os.path.join(OUT_DIR, f"magi_vars_{T}_ALL.csv")
        k_to_T.to_csv(vars_csv, index=False)
        print(f"[SAVED] Variables (k→T ALL) → {vars_csv}")

        # 4) build subgraph among {T} ∪ selected_k
        events_set = selected_k | {T}
        df_trim = _fetch_subgraph_by_targets(conn, sorted(events_set))
        df_trim = df_trim[
            df_trim["target_concept_code"].isin(events_set) &
            df_trim["concept_code"].isin(events_set)
        ].copy()

        # --- drop exact duplicate rows (all columns identical) ---
        before = len(df_trim)
        df_trim = df_trim.drop_duplicates()
        after = len(df_trim)
        if after < before:
            print(f"[DEDUP] subgraph exact-row duplicates removed: {before - after}  (kept {after})")

        # Recompute derived cols (incl. total_effect) on the subgraph
        df_trim = _ensure_derived_cols(df_trim)

        # Correct orientation in the prints:
        print(f"[TRIM] rows={len(df_trim):,}  events={len(events_set)}  "
              f"k→T rows={(df_trim['concept_code'] == T).sum()}  "
              f"T→j rows={(df_trim['target_concept_code'] == T).sum()}")

        # 5) save subgraph (for audit)
        sub_csv = os.path.join(OUT_DIR, f"magi_subgraph_{T}.csv")
        df_trim.to_csv(sub_csv, index=False)
        print(f"[SAVED] Subgraph → {sub_csv}")

        # 6) run MAGI
        try:
            res = analyze_causal_sequence_py(df_trim, events=None, name_map=None, force_outcome=T)
        except TypeError:
            res = analyze_causal_sequence_py(df_trim)
            outcome_used = res.get("order_used", [None])[-1]
            if outcome_used != T:
                print(f"[NOTE] outcome auto-inferred as {outcome_used}, not {T}")

        # 7) save NON-ZERO coefficients only
        outcome_used = res.get("order_used", [T])[-1]
        coef_df = res["coef_df"].copy()

        # robust: accept 'coef' or 'beta' column name
        coef_col = "coef" if "coef" in coef_df.columns else ("beta" if "beta" in coef_df.columns else None)
        if coef_col is None:
            raise KeyError("Coefficient column not found in coef_df (expected 'coef' or 'beta').")

        # filter to non-zero (tolerance to avoid tiny numerical noise)
        eps = 1e-12
        mask_nz = coef_df[coef_col].astype(float).abs() > eps
        coef_nz = coef_df.loc[mask_nz].copy()

        coef_csv = os.path.join(OUT_DIR, f"magi_coef_{outcome_used}_nonzero.csv")
        coef_nz.to_csv(coef_csv, index=False)

        print(f"[SAVED] Coefficients (non-zero only) → {coef_csv}  "
              f"| kept={len(coef_nz):,} of {len(coef_df):,}  "
              f"| antecedents={len(res.get('order_used', [])) - 1}  "
              f"| used_total_effect={res.get('used_total_effect', True)}")

# ========= main loop =========
uri = f"file:{MAGI_DB_PATH}?mode=ro"
with sqlite3.connect(uri, uri=True) as conn:
    for T in TARGETS:
        print("\n" + "=" * 100)
        print(f"[RUN] Target = {T}")

        # 1) k→T rows (concept_code == T)
        k_to_T = _fetch_k_to_T(conn, T)
        if k_to_T.empty:
            print(f"[WARN] No predictor→target rows for {T}; skipping.")
            continue

        # 2) derive totals/effects on the k→T list
        k_to_T = _ensure_derived_cols(k_to_T)

        # 3) select predictors: top-K by total_effect (one row per k)
        sel_rows = _select_top_by_te_unique_k(k_to_T, top_k=TOP_K)
        if sel_rows.empty:
            print(f"[WARN] No predictors selected for {T}; skipping.")
            continue

        # IMPORTANT: predictors are the LEFT side (target_concept_code)
        selected_k = set(sel_rows["target_concept_code"].astype(str))
        print(f"[SELECT] unique k available={k_to_T['target_concept_code'].nunique():,}  "
              f"selected={len(selected_k):,}")

        # Save the selected variables list for audit/import
        vars_csv = os.path.join(OUT_DIR, f"magi_vars_{T}_topByTE.csv")
        sel_rows.to_csv(vars_csv, index=False)
        print(f"[SAVED] Variables (k→T top-by-TE) → {vars_csv}")

        # 4) build subgraph among {T} ∪ selected_k
        events_set = selected_k | {T}
        df_trim = _fetch_subgraph_by_targets(conn, sorted(events_set))
        df_trim = df_trim[
            df_trim["target_concept_code"].isin(events_set) &
            df_trim["concept_code"].isin(events_set)
        ].copy()

        # Recompute derived cols (incl. total_effect) on the subgraph
        df_trim = _ensure_derived_cols(df_trim)

        # Correct orientation in the prints:
        #   k→T rows: concept_code == T
        #   T→j rows: target_concept_code == T
        print(f"[TRIM] rows={len(df_trim):,}  events={len(events_set)}  "
              f"k→T rows={(df_trim['concept_code'] == T).sum()}  "
              f"T→j rows={(df_trim['target_concept_code'] == T).sum()}")

        # 5) save subgraph (for audit)
        sub_csv = os.path.join(OUT_DIR, f"magi_subgraph_{T}.csv")
        df_trim.to_csv(sub_csv, index=False)
        print(f"[SAVED] Subgraph → {sub_csv}")

        # 6) run MAGI
        try:
            res = analyze_causal_sequence_py(df_trim, events=None, name_map=None, force_outcome=T)
        except TypeError:
            res = analyze_causal_sequence_py(df_trim)
            outcome_used = res.get("order_used", [None])[-1]
            if outcome_used != T:
                print(f"[NOTE] outcome auto-inferred as {outcome_used}, not {T}")

        # 7) save coefficients
        outcome_used = res.get("order_used", [T])[-1]
        coef_df = res["coef_df"]
        coef_csv = os.path.join(OUT_DIR, f"magi_coef_{outcome_used}.csv")
        coef_df.to_csv(coef_csv, index=False)
        print(f"[SAVED] Coefficients → {coef_csv}  | antecedents={len(res.get('order_used', [])) - 1}  "
              f"used_total_effect={res.get('used_total_effect', True)}")


## list all tables

uri = f"file:{MAGI_DB_PATH}?mode=ro"  # read-only; no writes
with sqlite3.connect(uri, uri=True) as conn:
    # 1) List tables and views
    tables = pd.read_sql_query("""
        SELECT name, type
        FROM sqlite_master
        WHERE type IN ('table','view')
        ORDER BY type, name
    """, conn)
    print("=== Objects in DB (tables & views) ===")
    print(tables.to_string(index=False))

    # 2) Helper to inspect a specific table (schema + sample rows)
    def peek(table: str, limit: int = 5):
        print(f"\n=== Schema: {table} ===")
        schema = pd.read_sql_query(f"PRAGMA table_info({table})", conn)
        print(schema.to_string(index=False))

        print(f"\n=== First {limit} rows: {table} ===")
        sample = pd.read_sql_query(f"SELECT * FROM {table} LIMIT {limit}", conn)
        print(sample)

    # Example: uncomment to inspect common tables
    # peek("concept_names", 5)
    # peek("magi_counts_filtered", 5)
    # peek("magi_counts_top500", 5)


## shape (rows, columns)
def table_shape(conn, table: str):
    # columns
    cols = pd.read_sql_query(f"PRAGMA table_info({table})", conn)
    n_cols = len(cols)
    # rows (COUNT(*) is the standard way in SQLite)
    n_rows = pd.read_sql_query(f"SELECT COUNT(*) AS n FROM {table}", conn).loc[0, "n"]
    return int(n_rows), int(n_cols)

uri = f"file:{MAGI_DB_PATH}?mode=ro"
with sqlite3.connect(uri, uri=True) as conn:
    for t in ["magi_counts_filtered", "magi_counts_top500"]:
        try:
            n_rows, n_cols = table_shape(conn, t)
            print(f"{t}: shape = ({n_rows:,} rows, {n_cols} cols)")
        except Exception as e:
            print(f"{t}: {e}")


uri = f"file:{MAGI_DB_PATH}?mode=ro"
with sqlite3.connect(uri, uri=True) as conn:
    df5 = pd.read_sql_query("""
      SELECT m.*,
             tcn.concept_code AS target_concept_code,
             ccn.concept_code AS concept_code
      FROM magi_counts_top500 m              -- change to magi_counts / magi_counts_filtered if you want
      JOIN concept_names tcn ON m.target_concept_code_int = tcn.concept_code_int
      JOIN concept_names ccn ON m.concept_code_int        = ccn.concept_code_int
      LIMIT 5
    """, conn)

print(df5)         # top 5 rows
print(df5.columns) # columns present
